In [3]:
from py2neo import Graph, Node, Relationship, NodeMatcher
from pandas import DataFrame

In [4]:
neo4j_config = { 'user': 'neo4j', 'password': 'neo4jneo4j', 'host': 'mylocalhost'}
# Connect to Graph
graph = Graph(**neo4j_config)
# Check that we are connected to the expected Neo4J Store - to avoid accidents...
dbname = graph.database.name
print(dbname)

egypts.db


## Number of nodes in the network

In [7]:
query = "match (n) return count(n) as cnt"
cur = graph.run(query)
dd = cur.data()
print(dd[0]["cnt"])

25178


## Components in the network

In [8]:
query = """
call algo.unionFind.stream('Person', 'contacts', {})
yield nodeId, setId
return count(distinct setId) as components
"""
cur = graph.run(query)
dd = cur.data()
print(dd[0]["components"])

3122


## Size of Top 20 Components

In [10]:
query = """
call algo.unionFind.stream('Person', 'contacts', {})
yield nodeId, setId
return setId, count(*) as component_size
order by component_size desc
limit 20
"""
cur = graph.run(query)
dd = cur.data()
for res in dd:
    print("Size: {size}".format(size=res["component_size"]))

Size: 17762
Size: 64
Size: 16
Size: 16
Size: 14
Size: 13
Size: 11
Size: 11
Size: 10
Size: 10
Size: 10
Size: 10
Size: 10
Size: 9
Size: 9
Size: 9
Size: 9
Size: 9
Size: 9
Size: 9
